In [1]:
import os
import shutil
import uuid
import xml.etree.ElementTree as ET

import pandas as pd
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [6]:
# Пути для сохранения разделенных выборок
images_text_detector_train_dir = "../../data/processed/3 Production/text_detector/train/images"
labels_text_detector_train_dir = "../../data/processed/3 Production/text_detector/train/labels"

images_text_detector_valid_dir = "../../data/processed/3 Production/text_detector/valid/images"
labels_text_detector_valid_dir = "../../data/processed/3 Production/text_detector/valid/labels"

images_text_detector_test_dir = "../../data/processed/3 Production/text_detector/test/images"
labels_text_detector_test_dir = "../../data/processed/3 Production/text_detector/test/labels"

images_text_recognizer_train_dir = "../../data/processed/3 Production/text_recognizer/train"

images_text_recognizer_valid_dir = "../../data/processed/3 Production/text_recognizer/valid"

images_text_recognizer_test_dir = "../../data/processed/3 Production/text_recognizer/test"


# Создание каталогов для train, valid, test и images
os.makedirs(images_text_detector_train_dir, exist_ok=True)
os.makedirs(labels_text_detector_train_dir, exist_ok=True)

os.makedirs(images_text_detector_valid_dir, exist_ok=True)
os.makedirs(labels_text_detector_valid_dir, exist_ok=True)

os.makedirs(images_text_detector_test_dir, exist_ok=True)
os.makedirs(labels_text_detector_test_dir, exist_ok=True)

os.makedirs(images_text_recognizer_train_dir, exist_ok=True)
os.makedirs(images_text_recognizer_valid_dir, exist_ok=True)
os.makedirs(images_text_recognizer_test_dir , exist_ok=True)


# Список для хранения путей к изображениям и разметкам
images = []
annotations = []
# 0 - Губернаторские отчёты
# 1 - Уставные грамоты – Афанасенков
# 2 - Уставные грамоты в jpg (Просветов)
labels = []

###############################################################################
############# Формируем датасет из каталога Губернаторские отчеты #############
###############################################################################

# Путь к каталогу с данными
data_dir = "../../data/raw/Распознавание текстов/Губернаторские отчеты"

# Перебор всех каталогов и файлов внутри data_dir
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".JPG"):
            image_path = os.path.join(root, file)
            annotation_path = os.path.join(root, file.replace(".JPG", "_pvoc_imglab.xml"))
            if os.path.exists(annotation_path):
                images.append(image_path)
                annotations.append(annotation_path)
                labels.append(0)

########################################################################################
############# Формируем датасет из каталога Уставные грамоты – Афанасенков #############
########################################################################################

# Путь к каталогу с данными
data_dir = "../../data/raw/Распознавание текстов/Уставные грамоты – Афанасенков"
# Перебор всех каталогов и файлов внутри data_dir
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            annotation_path = os.path.join(root, file.replace(".jpg", "_pvoc_imglab.xml"))
            if os.path.exists(annotation_path):
                images.append(image_path)
                annotations.append(annotation_path)
                labels.append(1)

############################################################################################
############# Формируем датасет из каталога Уставные грамоты в jpg (Просветов) #############
############################################################################################

image_dir = "../../data/raw/Распознавание текстов/Уставные грамоты в jpg (Просветов)"
annotation_dir = "../../data/raw/Распознавание текстов/Уставные грамоты в jpg (Просветов)/Обработка/Просветов (13.12)"

# Перебор всех файлов изображений в image_dir
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            annotation_file = file.replace(".jpg", "_pvoc_imglab.xml")
            annotation_path = os.path.join(annotation_dir, annotation_file)
            if os.path.exists(annotation_path):
                images.append(image_path)
                annotations.append(annotation_path)
                labels.append(2)


# Разделение выборки на train, valid и test
train_images, test_images, train_annotations, test_annotations, train_labels, test_labels = train_test_split(images, annotations, labels, test_size=0.2, random_state=42)
train_images, valid_images, train_annotations, valid_annotations, train_labels, valid_labels = train_test_split(train_images, train_annotations, train_labels, test_size=0.2, random_state=42)

# Функция для копирования файлов
def copy_files(images, annotations, dest_dir):
    for image, annotation in zip(images, annotations):
        shutil.copy(image, dest_dir)
        shutil.copy(annotation, dest_dir)

# Функция для преобразования координат в относительные координаты
def convert_coordinates(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2]) / 2.0
    y = (box[1] + box[3]) / 2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    x_rel = x * dw
    w_rel = w * dw
    y_rel = y * dh
    h_rel = h * dh
    return x_rel, y_rel, w_rel, h_rel

# Функция для записи разметки в формате YOLO
def write_yolo_annotation(annotation_path, size, objects, output_file):
    with open(output_file, "w") as f:
        for obj in objects:
            x_rel, y_rel, w_rel, h_rel = convert_coordinates(size, obj["bbox"])
            # первый 0 - единственный класс разметки 'text'
            f.write(f"0 {x_rel} {y_rel} {w_rel} {h_rel}\n")

# Преобразование разметки для train выборки
def process_data(
        images, annotations, labels,
        text_detector_images_dir, text_detector_labels_dir,
        text_recognizer_images_dir,
        data_type
):
    
    # Датафрейм для датасета
    data = pd.DataFrame()
    
    err_count = 0

    for image_path, annotation_path, dir_label in tqdm(zip(images, annotations, labels), total=len(images), desc=f"Подготавливаем {data_type} выборку"):
        # Копирование изображений в директорию images
        image_name = os.path.basename(image_path)

        shutil.copy(image_path, text_detector_images_dir + "/" + image_name)

        ###### Преобразование разметки в формат YOLO
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        txt_filename = image_name.replace(".JPG", ".txt").replace(".jpg", ".txt")
        txt_path = text_detector_labels_dir + "/" + txt_filename

        size = (int(root.find("size/width").text), int(root.find("size/height").text))
        objects = []
        for obj in root.findall("object"):
            name = obj.find("name").text
            bbox = [
                float(obj.find("bndbox/xmin").text),
                float(obj.find("bndbox/ymin").text),
                float(obj.find("bndbox/xmax").text),
                float(obj.find("bndbox/ymax").text),
            ]
            objects.append({"name": name, "bbox": bbox})

        write_yolo_annotation(annotation_path, size, objects, txt_path)
        
        ###### Преобразования разметки в формат TrOCR
        
        objects = []
        
        for row_num, obj in enumerate(root.findall("object")):
            segment_name = image_name.split(".")[0] + "___" + str(row_num) + ".JPG"
            name = obj.find("name").text
            bbox = [
                float(obj.find("bndbox/xmin").text),
                float(obj.find("bndbox/ymin").text),
                float(obj.find("bndbox/xmax").text),
                float(obj.find("bndbox/ymax").text),
            ]
            
            img = Image.open(image_path)
            img = ImageOps.exif_transpose(img)
            
            try:
                cropped_segment = img.crop(bbox)
                cropped_segment.save(os.path.join(text_recognizer_images_dir, segment_name))
                
                objects.append({"file_name": segment_name, "text": name, "label": dir_label})
                
            # могут быть ошибки разметки и DecompressionBombError
            except Exception:
                err_count += 1
                continue
        
        # Добавляем полученные данные в датафрейм 
        new_data = pd.DataFrame(data=objects)
        data = pd.concat([data, new_data])
    
    return data

In [7]:
from joblib import Parallel, delayed
import numpy as np

def chunkify(lst, n):
    """Split lst into n chunks."""
    return np.array_split(lst, n)

def run_in_parallel(
        images, annotations, labels,
        text_detector_images_dir, text_detector_labels_dir,
        text_recognizer_images_dir,
        data_type, n_jobs):
    
    image_chunks = chunkify(images, n_jobs)
    annotation_chunks = chunkify(annotations, n_jobs)
    
    tasks = zip(image_chunks, annotation_chunks, [labels] * n_jobs, [text_detector_images_dir] * n_jobs, [text_detector_labels_dir] * n_jobs, [text_recognizer_images_dir] * n_jobs, [data_type] * n_jobs)
    
    # Use Joblib's Parallel and delayed to run process_data in parallel across chunks
    results = Parallel(n_jobs=n_jobs)(delayed(process_data)(*args) for args in tqdm(tasks))
    
    # Concatenate the results from each chunk
    final_dataframe = pd.concat(results, ignore_index=True)
    
    return final_dataframe

In [8]:
n_jobs = 8

train_dataframe = run_in_parallel(
    train_images, train_annotations, train_labels,
    images_text_detector_train_dir, labels_text_detector_train_dir,
    images_text_recognizer_train_dir,
    "обучающую", n_jobs
)

valid_dataframe = run_in_parallel(
    valid_images, valid_annotations, valid_labels,
    images_text_detector_valid_dir, labels_text_detector_valid_dir, 
    images_text_recognizer_valid_dir,
    "валидационную", n_jobs
)

test_dataframe = run_in_parallel(
    test_images, test_annotations, test_labels,
    images_text_detector_test_dir, labels_text_detector_test_dir,
    images_text_recognizer_test_dir,
    "тестовую", n_jobs
)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [9]:
train_dataframe.to_csv("../../data/processed/3 Production/train.csv")
valid_dataframe.to_csv("../../data/processed/3 Production/valid.csv")
test_dataframe.to_csv("../../data/processed/3 Production/test.csv")